In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_log_error 

In [2]:
df = pd.read_csv("../Data/Fixed-Fixed.csv")
df.head()

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.0,0,0,1,52.976,18.084,9.0351
1,0.0,0,0,2,145.920,17.293,9.3771
2,0.0,0,0,3,285.990,17.430,9.5803
3,0.0,0,0,4,323.400,28.366,10.8720
4,0.0,0,0,5,472.810,17.537,9.7190


In [3]:
seed=14

# Preprocessing

In [4]:
X = df[["depth", "l1", "l2", "Mode"]].values
Y = df[["Frequency", "Max_Deflection", "Average_Deflection"]].values

In [5]:
# Split the data

_X_train, _X_test, _y_train, _y_test = train_test_split(X, Y, test_size=0.30, random_state=seed)

In [6]:
# Standard Scalar

std_X = StandardScaler()
std_X.fit(_X_train[:, :-1]) # Fitting on the Numerical Part of the Data

std_Y = StandardScaler()
std_Y.fit(_y_train) # Fitting on the Numerical Part of the Data

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
get_feature = lambda x: np.hstack((std_X.transform(x[:, :-1]), np.eye(6)[x[:, -1].astype(np.int8) - 1]))

In [8]:
get_train_output = lambda y: std_Y.transform(y)
get_output = lambda y: std_Y.inverse_transform(y)

# Data

In [9]:
X_train = get_feature(_X_train)
y_train = get_train_output(_y_train)

In [10]:
X_test = get_feature(_X_test)
y_test = _y_test

# Support Vector Regressor Method 1

### 1. Training

In [11]:
svr = SVR()

In [12]:
svr.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [13]:
parametrs = {
    'C': [0.5, 1.0, 2.0],
    'degree': [2, 3, 4],
    'kernel': ['rbf', 'poly'],
    'epsilon': [0.001, 0.1, 1.0]
}

In [14]:
model1 = GridSearchCV(svr, parametrs)
model2 = GridSearchCV(svr, parametrs)
model3 = GridSearchCV(svr, parametrs)

In [15]:
model1.fit(X_train, y_train[:,0])

GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.5, 1.0, 2.0], 'degree': [2, 3, 4],
                         'epsilon': [0.001, 0.1, 1.0],
                         'kernel': ['rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [16]:
# Best Parametrs

model1.best_params_

{'C': 1.0, 'degree': 2, 'epsilon': 0.001, 'kernel': 'poly'}

In [17]:
model2.fit(X_train, y_train[:,1])

GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.5, 1.0, 2.0], 'degree': [2, 3, 4],
                         'epsilon': [0.001, 0.1, 1.0],
                         'kernel': ['rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [18]:
# Best Parametrs

model2.best_params_

{'C': 1.0, 'degree': 2, 'epsilon': 0.001, 'kernel': 'poly'}

In [19]:
model3.fit(X_train, y_train[:,2])

GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.5, 1.0, 2.0], 'degree': [2, 3, 4],
                         'epsilon': [0.001, 0.1, 1.0],
                         'kernel': ['rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [20]:
# Best Parametrs

model3.best_params_

{'C': 2.0, 'degree': 2, 'epsilon': 0.1, 'kernel': 'rbf'}

### 2. Testing

In [21]:
res1 = model1.predict(X_test)
res2 = model2.predict(X_test)
res3 = model3.predict(X_test)

y_pred_mode1 = get_output(np.hstack((res1.reshape(-1, 1), res2.reshape(-1, 1), res3.reshape(-1, 1))))

In [22]:
assert y_pred_mode1.shape == y_test.shape

### 3. Evaluation

In [23]:
# Frequency Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(y_pred_mode1[:,0], y_test[:,0])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(y_pred_mode1[:,0], y_test[:,0])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(y_pred_mode1[:,0], y_test[:,0])))

Explained Varaince Score: 99.993539%
Mean Absolute Error: 0.708895
Mean Squared Error: 0.000018


In [24]:
# Max Deflection Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(y_pred_mode1[:,1], y_test[:,1])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(y_pred_mode1[:,1], y_test[:,1])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(y_pred_mode1[:,1], y_test[:,1])))

Explained Varaince Score: 99.951406%
Mean Absolute Error: 0.048013
Mean Squared Error: 0.000024


In [25]:
# Average Defelction Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(y_pred_mode1[:,2], y_test[:,2])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(y_pred_mode1[:,2], y_test[:,2])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(y_pred_mode1[:,2], y_test[:,2])))

Explained Varaince Score: 70.553163%
Mean Absolute Error: 0.313308
Mean Squared Error: 0.001599


# Support Vector Regressor Method 2

### 1. Training

In [26]:
clf = MultiOutputRegressor(SVR())

In [27]:
clf.get_params()

{'estimator__C': 1.0,
 'estimator__cache_size': 200,
 'estimator__coef0': 0.0,
 'estimator__degree': 3,
 'estimator__epsilon': 0.1,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
     kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
 'n_jobs': None}

In [28]:
parametrs = {
    'estimator__C': [0.5, 1.0, 2.0],
    'estimator__degree': [2, 3, 4],
    'estimator__kernel': ['rbf', 'poly'],
    'estimator__epsilon': [0.001, 0.1, 1.0]
}

In [29]:
model4 = GridSearchCV(clf, parametrs)
model4.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=MultiOutputRegressor(estimator=SVR(C=1.0, cache_size=200,
                                                          coef0=0.0, degree=3,
                                                          epsilon=0.1,
                                                          gamma='scale',
                                                          kernel='rbf',
                                                          max_iter=-1,
                                                          shrinking=True,
                                                          tol=0.001,
                                                          verbose=False),
                                            n_jobs=None),
             iid='deprecated', n_jobs=None,
             param_grid={'estimator__C': [0.5, 1.0, 2.0],
                         'estimator__degree': [2, 3, 4],
                         'estimator__epsilon': [0.001, 0.1, 1.0],
              

In [30]:
model4.best_params_

{'estimator__C': 2.0,
 'estimator__degree': 2,
 'estimator__epsilon': 0.001,
 'estimator__kernel': 'rbf'}

### 2. Validation

In [31]:
res4 = model4.predict(X_test)
y_pred_mode2 = get_output(res4)

In [32]:
assert y_pred_mode2.shape == y_test.shape

### 3. Evaluation

In [33]:
# Requency Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(y_pred_mode2[:,0], y_test[:,0])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(y_pred_mode2[:,0], y_test[:,0])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(y_pred_mode2[:,0], y_test[:,0])))

Explained Varaince Score: 99.967723%
Mean Absolute Error: 1.562724
Mean Squared Error: 0.001100


In [34]:
# Max Deflection Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(y_pred_mode2[:,1], y_test[:,1])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(y_pred_mode2[:,1], y_test[:,1])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(y_pred_mode2[:,1], y_test[:,1])))

Explained Varaince Score: 99.695230%
Mean Absolute Error: 0.088277
Mean Squared Error: 0.000084


In [35]:
# Average Defelction Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(y_pred_mode2[:,2], y_test[:,2])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(y_pred_mode2[:,2], y_test[:,2])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(y_pred_mode2[:,2], y_test[:,2])))

Explained Varaince Score: 69.916094%
Mean Absolute Error: 0.316619
Mean Squared Error: 0.001639


# Final Testing Phase

In this phase separeate simulations were done with new set of points generated random;y with different random seed.

In [36]:
df_test = pd.read_csv("../Data/Fixed-Fixed-Test.csv")
df_test.head()

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.5,63,510,1,52.613,7.5540,3.8030
1,0.5,63,510,2,144.790,7.2355,3.6191
2,0.5,63,510,3,283.540,7.3149,4.0421
3,0.5,63,510,4,322.530,11.8550,4.5453
4,0.5,63,510,5,468.260,7.4041,3.8655


In [37]:
test_X = df_test[["depth", "l1", "l2", "Mode"]].values
test_X = get_feature(test_X)

test_Y = df_test[["Frequency", "Max_Deflection", "Average_Deflection"]].values

### Mode 1

In [38]:
test_res1 = model1.predict(test_X)
test_res2 = model2.predict(test_X)
test_res3 = model3.predict(test_X)

test_pred_y_mode_1 = get_output(np.hstack((test_res1.reshape(-1, 1), test_res2.reshape(-1, 1), test_res3.reshape(-1, 1))))

In [39]:
assert test_pred_y_mode_1.shape == test_Y.shape

In [40]:
# Fequency Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(test_pred_y_mode_1[:,0], test_Y[:,0])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(test_pred_y_mode_1[:,0], test_Y[:,0])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(test_pred_y_mode_1[:,0], test_Y[:,0])))

Explained Varaince Score: 99.980973%
Mean Absolute Error: 0.943772
Mean Squared Error: 0.000040


In [41]:
# Max Deflection Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(test_pred_y_mode_1[:,1], test_Y[:,1])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(test_pred_y_mode_1[:,1], test_Y[:,1])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(test_pred_y_mode_1[:,1], test_Y[:,1])))

Explained Varaince Score: 66.205908%
Mean Absolute Error: 11.318089
Mean Squared Error: 0.644649


In [42]:
# Average Defelction Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(test_pred_y_mode_1[:,2], test_Y[:,2])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(test_pred_y_mode_1[:,2], test_Y[:,2])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(test_pred_y_mode_1[:,2], test_Y[:,2])))

Explained Varaince Score: 60.161892%
Mean Absolute Error: 5.819700
Mean Squared Error: 0.571772


### Mode 2

In [43]:
test_res4 = model4.predict(test_X)
test_pred_y_mode_2 = get_output(test_res4)

In [44]:
assert test_pred_y_mode_2.shape == test_Y.shape

In [45]:
# Fequency Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(test_pred_y_mode_2[:,0], test_Y[:,0])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(test_pred_y_mode_2[:,0], test_Y[:,0])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(test_pred_y_mode_2[:,0], test_Y[:,0])))

Explained Varaince Score: 99.967733%
Mean Absolute Error: 1.559183
Mean Squared Error: 0.000453


In [46]:
# Max Deflection Outcome

print("Explained Varaince Score: {:0.6f}%".format(100*explained_variance_score(test_pred_y_mode_2[:,1], test_Y[:,1])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(test_pred_y_mode_2[:,1], test_Y[:,1])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(test_pred_y_mode_2[:,1], test_Y[:,1])))

Explained Varaince Score: 66.267247%
Mean Absolute Error: 11.315369
Mean Squared Error: 0.644507


In [47]:
# Average Defelction Outcome

print("Explained Variance Score: {:0.6f}%".format(100*explained_variance_score(test_pred_y_mode_2[:,2], test_Y[:,2])))
print("Mean Absolute Error: {:0.6f}".format(mean_absolute_error(test_pred_y_mode_2[:,2], test_Y[:,2])))
print("Mean Squared Error: {:0.6f}".format(mean_squared_log_error(test_pred_y_mode_2[:,2], test_Y[:,2])))

Explained Variance Score: 59.772937%
Mean Absolute Error: 5.816237
Mean Squared Error: 0.571219
